In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

df = pd.read_excel('data/half_binary_01_vectors.xlsx')
df.head(10)

C:\Users\Sorra\AppData\Local\Temp\ipykernel_12452\2156475923.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Rate,Episodes_scaled,Episode length_normalized,TV Сериал,Фильм,OVA,TV Спецвыпуск,ONA,Спецвыпуск,PG-13,R-17,R+,PG,G,Не указано,Приключения,Сёдзё,Спорт,Сёнен,Сверхъестественное,Романтика,Тайна,Сэйнэн,Комедия,Детское,Гурман,Ужасы,Фантастика,Авангард,Экшен,Этти,Фэнтези,Драма,Повседневность,Дзёсей,Триллер,Образовательное,Кроссдрессинг,Гонки,Реверс-гарем,Исполнительское искусство,Реинкарнация,Пародия,Забота о детях,Идолы (Жен.),Исторический,Исэкай,Стратегические игры,Удостоено наград,Спортивные единоборства,Гарем,Меха,Питомцы,Выживание,Вампиры,Супер сила,Самураи,CGDCT,Музыка,Школа,Иясикэй,Антропоморфизм,Хулиганы,Гэг-юмор,Военное,Жестокость,Изобразительное искусство,Командный спорт,Путешествие во времени,Любовный многоугольник,Шоу-бизнес,Работа,Магическая смена пола,Психологическое,Организованная преступность,Культура отаку,Мифология,Идолы (Муж.),Игра с высокими ставками,Боевые искусства,Романтический подтекст,Медицина,Махо-сёдзё,Детектив,Видеоигры,Взрослые персонажи,Космос,наличие Ваншота,наличие Додзинси,наличие TV Сериала или ONA,наличие Фильма,наличие Ранобэ или Новеллы,наличие Маньхуа,наличие Манхвы,наличие Манги,Kafka,LIDENFILMS,Delight,Public &amp; Basic,Venet,PP Project,Revoroot,Makaria,Decovocal,Daume,Robot Communications,Himeyuri Alumnae Incorporated Foundation,Brio,Shogakukan Music & Digital,Kuri Jikken Manga Koubou,Mir,Indivision,Karaku,Dentsu Inc.,Dyna Method,JCF,Usagi Ou,Suna Kouhou,Magma,asread.,Fuji TV,Sanctuary,Production Reed,Saigo no Shudan,Picograph,Mook,Ultra Super Pictures,Dadashow,VROOOOM,Magia Doraglier,Marine,Yaoyorozu,Ishikawa Pro,SBS TV,Maikaze,UGOKI,Victor,DLE,Kingyoiro,Felix Film,ASK Animation,Takara,MASTER LIGHTS,Media Castle,Shueisha,Rising Force,501,Issen,The National Film Center Japan,MAT,Mippei Eigeki Kiryuukan,General,CompTown,Toho,Windy,A-Line,Coa,Khronos,helo.inc,GoHands,Production +h.,AZ Creative,Comet,Grouper,Sunrise Beyond,Cyclone Graphics,GARDEN LODGE,M.S.C,dwarf,flying DOG,Xiaoming Taiji,Gravity Well,Charaction,NYAV Post,Sotsu,Junior,Takeshobo,Rocen,J.C.Staff,Tele-Cartoon Japan,Nelvana,Heart &amp; Soul,Plus Heads,Think Corporation,SIDO LIMITED,Haoliners Animation League,M2,Liber,Ponoc,Heewon,Shin-Ei,Planet Cartoon,Nomad,Xing Yi Kai Chen,Ripromo,OLM,KeyEast,Oxybot,Anon Pictures,Digital Media Lab,Egg Firm,Typhoon Graphics,Trinet,Kanaban Graphics,Changchun Unity of Knowledge and Action,Live2D Creative,ufotable,VOLN,Ai ga areba Daijobu,AION,Saber Project,TUBA,Tsuchida,TriF,Broccoli,Stingray,Yokohama Animation Lab,Wonder Cat,MontBlanc Pictures,production doA,Oddjob,TthunDer,Nice Boat,Itasca,Tatsunoko,Village,team Yamahitsuji,Wit,Toho Interactive,Ordet,Tezuka,Kazami Gakuen Koushiki Douga-bu,Yi Chen,Joicy,Fifth Avenue,Quebico,domerica,The Monk,CGCG,Chaos Project,Aubec,G&amp;G,DAX,P.I.C.S.,VCRWORKS,Sublimation,Group TAC,Blue bread,Mushi,Akatsuki,Mokai Technology,Daiei Film Co. Ltd.,LICO,Graphinica,Yomiuri Telecasting Corporation,BUG FILMS,Milky Cartoon,Hotline,Katsudou-manga-kan,Nakamura,TBS,Agent 21,Sharefun,L²Studio,Urban Product,Arch,7doc,Meruhensha,Shirogumi,TMS,Enoki Films,Viz Media,Gonzo,Animaruya,Asura Film,Ai Si Animation,XFLAG Pictures,NHK Enterprises,Original Force,Matrix,Gyorai Eizo Inc.,Kawamoto Productions Ltd.,Gazelle,Banpresto,Passione,ARCUS,StudioRF Inc.,Quad,Gambit,Square Pictures,Bridge,Blaze,Eiken,Zexcs,OB Planning,Tecarat,HOTZIPANG,PuYUKAI,Iyasakadou Film,Bouncy,White Fox,Telescreen,ILCA,Aikikaku Center,Tokuma Japan,Wako,Alfred Imageworks,Story Effect,Monster's Egg,Signal.MD,Jemi,Office Nobu,Hong Ying,GIFTanimation,Bakken Record,Dongwoo A&E,CMC Media,Dynamic Planning,Okuruto Noboru,LandQ,SILVER LINK.,Marza Animation Planet,PPM,Konami,Live,Kitty Film Mitaka,Signpost,KIDS STATION,Actas,Craftar,B&amp;T,Guts,UM,Unend,Sparky,miHoYoAnime,Dibetagurashi Production Committee,MOTHER,Marone,Yamato Works,Omnibus Japan,SamBakZa,Creators Dot Com,October Media,Jichitai Anime,FILMONY,Sota,AIC Classic,Sunwoo,Daisy,Imagica,SAMG,Future Planet,Triple X,HuaMei,Artmic,Lingsanwu,Shogakukan,Flint Sugar,

In [2]:
df.shape

(12191, 986)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12191 entries, 0 to 12190
Columns: 986 entries, Rate to Lerche
dtypes: float64(3), int64(983)
memory usage: 91.7 MB


In [4]:
import numpy as np
import autokeras as ak
from sklearn.model_selection import train_test_split


Using TensorFlow backend


In [5]:
X = df.drop(columns=['Rate'])  # Признаки, исключая первый столбец
y = df['Rate']  # Целевая переменная (рейтинг)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Установка test_size в 0.2 для разделения на обучающий и тестовый наборы

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)  # Установка test_size в 0.1 для разделения на обучающий и валидационный наборы

In [6]:
# Проверка размерностей массивов
print("Размерности массивов:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_val:", X_test.shape)
print("y_val:", y_test.shape)

Размерности массивов:
X_train: (8776, 985)
y_train: (8776,)
X_val: (2439, 985)
y_val: (2439,)


In [7]:
import autokeras as ak

# Инициализация регрессора AutoKeras
reg = ak.StructuredDataRegressor(
    max_trials=20,  # Максимальное количество различных моделей, которые могут быть протестированы
    overwrite=True,  # Перезаписывать результаты предыдущих обучений
    loss='mean_squared_error',
    metrics=['mean_absolute_error'],  # Оптимизируемая метрика (в данном случае, минимизация потерь)
    project_name="model_01",  # Название проекта (для сохранения результатов)
)

# Обучение регрессора на обучающих данных
reg.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val), verbose=1)

# Сохранение лучшей модели
best_model = reg.export_model()
best_model.save('best_autokeras_model')
best_model.save('best_model3', save_format='tf')

Trial 20 Complete [00h 33m 25s]
val_loss: 0.40668365359306335

Best val_loss So Far: 0.3663184642791748
Total elapsed time: 09h 26m 22s
Epoch 1/30
275/275 [==============================] - 67s 206ms/step - loss: 4.9406 - mean_absolute_error: 1.6763 - val_loss: 32.2778 - val_mean_absolute_error: 5.6207
Epoch 2/30
275/275 [==============================] - 49s 178ms/step - loss: 1.4478 - mean_absolute_error: 0.9444 - val_loss: 2.2194 - val_mean_absolute_error: 1.3321
Epoch 3/30
275/275 [==============================] - 48s 173ms/step - loss: 1.1671 - mean_absolute_error: 0.8541 - val_loss: 0.4504 - val_mean_absolute_error: 0.5346
Epoch 4/30
275/275 [==============================] - 47s 172ms/step - loss: 0.9838 - mean_absolute_error: 0.7836 - val_loss: 0.4582 - val_mean_absolute_error: 0.5433
Epoch 5/30
275/275 [==============================] - 47s 172ms/step - loss: 0.8241 - mean_absolute_error: 0.7161 - val_loss: 0.8762 - val_mean_absolute_error: 0.7739
Epoch 6/30
275/275 [========

In [8]:
from tensorflow.keras.models import load_model
loaded_model = load_model('best_autokeras_model')
loaded_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 985)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 985)              0         
 ltiCategoryEncoding)                                            
                                                                 
 dense (Dense)               (None, 64)                63104     
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 64)                0         
                                                                 
 dropout (Dropout)           (None, 64)                0     

In [9]:
# Оценка модели на тестовых данных
loss, accuracy = loaded_model.evaluate(X_test, y_test)

# Вывод результатов
print("Loss:", loss)
print("Mean Absolute Error:", accuracy)

77/77 [==============================] - 22s 174ms/step - loss: 0.3931 - mean_absolute_error: 0.4914
Loss: 0.3930981457233429
Mean Absolute Error: 0.49140551686286926


In [10]:
df_test = pd.read_excel('data/half_binary_01_test_vector.xlsx')
X = df_test

# Загрузка обученной модели
loaded_model = load_model('best_autokeras_model')

# Предсказание на тестовых данных
predictions = loaded_model.predict(df_test)

# Вывод результатов предсказания
print(predictions)

1/1 [==============================] - 9s 9s/step
[[6.841044]]
